In [0]:
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/industry_energy/energy_train.csv
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/industry_energy/energy_test.csv
!wget https://raw.githubusercontent.com/LuisM78/Appliances-energy-prediction-data/master/energydata_complete.csv

In [0]:
import pandas as pd

df = pd.read_csv('energydata_complete.csv')
trdf = pd.read_csv('energy_train.csv')
tedf = pd.read_csv('energy_test.csv')

# Drop nulls and date column
df = df.dropna() 
dataset = df.drop('date', axis=1)

In [0]:
columns = dataset.columns

dataset = dataset.round(1)

In [0]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np 
values = dataset.values # should return a list 
# This estimator standardizes features by removing the mean and scaling to unit variance such that the values are centered around 0 
scaler = StandardScaler()
scaled = scaler.fit_transform(values)
# Create features and label dataset. NOTE: We're predicting a value 24 hours into the future. 
X = scaled[:][:-144] # All of the above columns and remove readings from last 24 hours
y = scaled[:, 0][144:] # Appliances or the target/label column and remove readings from first 24 hours
from sklearn.model_selection import train_test_split
# split into train and test sets
trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.20, random_state=42, shuffle = False)
# Create overlapping windows of lagged values for training and testing datasets
timesteps = 864
train_generator = TimeseriesGenerator(trainX, trainY, length=timesteps, sampling_rate=1, batch_size=timesteps)
test_generator = TimeseriesGenerator(testX, testY, length=timesteps, sampling_rate=1, batch_size=timesteps)
train_X, train_y = train_generator[0]
test_X, test_y = test_generator[0]
train_samples = train_X.shape[0]*len(train_generator)
test_samples = test_X.shape[0]*len(test_generator)
print("Total Records (n): {}".format(df.count()))
print("Total Records after adjusting for 24 hours: {}".format(len(X)))
print("Number of samples in training set (.8 * n): trainX = {}".format(trainX.shape[0]))
print("Number of samples in testing set (.2 * n): testX = {}".format(testX.shape[0]))
print("Size of individual batches: {}".format(test_X.shape[1]))
print("Number of total samples in training feature set: {}".format(train_samples))
print("Number of samples in testing feature set: {}".format(test_samples))

In [0]:
%tensorflow_version 2.x

In [0]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping
units = 128
num_epoch = 1000
learning_rate = 0.00144
model = Sequential()
model.add(LSTM(units, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(LeakyReLU(alpha=0.5)) 
model.add(Dropout(0.1))
model.add(Dense(1))
adam = Adam(lr=learning_rate)
# Stop training when a monitored quantity has stopped improving.
callback = [EarlyStopping(monitor="loss", min_delta = 0.00001, patience = 50, mode = 'auto', restore_best_weights=True)] 
# Using regression loss function 'Mean Standard Error' and validation metric 'Mean Absolute Error'
model.compile(loss='mse', optimizer=adam, metrics=['mae'])
history = model.fit_generator(train_generator, epochs=num_epoch, validation_data=test_generator, callbacks = callback, verbose=2,
                              shuffle=False, initial_epoch=0)


In [0]:
#TRAIN_on_WHOLE_DATA
#PREDICT on new data
model.save('DQ-EP12.h5')

from tensorflow.keras.models import load_model

dqmodel = load_model('DQ-EP12.h5')

In [28]:
dqmodel.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               80384     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 80,513
Trainable params: 80,513
Non-trainable params: 0
_________________________________________________________________


In [0]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np 
values = dataset.values
scaler = StandardScaler()
scaled = scaler.fit_transform(values)
X = scaled[:]
y = scaled[:, 0]
timesteps = 864
train_generator = TimeseriesGenerator(X, y, length=timesteps, sampling_rate=1, batch_size=timesteps)
train_X, train_y = train_generator[0]
train_samples = train_X.shape[0]*len(train_generator)

In [0]:
callback = [EarlyStopping(monitor="loss", min_delta = 0.00001, patience = 50, mode = 'auto', restore_best_weights=True)]
history = dqmodel.fit_generator(train_generator, epochs=1000, callbacks = callback, verbose=2, shuffle=False, initial_epoch=0)

Epoch 1/1000
22/22 - 5s - loss: 1.0565 - mean_absolute_error: 0.5285
Epoch 2/1000
22/22 - 5s - loss: 0.5986 - mean_absolute_error: 0.3922
Epoch 3/1000
22/22 - 5s - loss: 0.5177 - mean_absolute_error: 0.3935
Epoch 4/1000
22/22 - 5s - loss: 0.4817 - mean_absolute_error: 0.3514
Epoch 5/1000
22/22 - 5s - loss: 0.4651 - mean_absolute_error: 0.4035
Epoch 6/1000
22/22 - 5s - loss: 0.4792 - mean_absolute_error: 0.3609
Epoch 7/1000
22/22 - 5s - loss: 0.4715 - mean_absolute_error: 0.4554
Epoch 8/1000
22/22 - 5s - loss: 0.4728 - mean_absolute_error: 0.3579
Epoch 9/1000
22/22 - 5s - loss: 0.4328 - mean_absolute_error: 0.4195
Epoch 10/1000
22/22 - 5s - loss: 0.3949 - mean_absolute_error: 0.3088
Epoch 11/1000
22/22 - 5s - loss: 0.3647 - mean_absolute_error: 0.3458
Epoch 12/1000
22/22 - 5s - loss: 0.3434 - mean_absolute_error: 0.2896
Epoch 13/1000
22/22 - 5s - loss: 0.3247 - mean_absolute_error: 0.3114
Epoch 14/1000
22/22 - 5s - loss: 0.3135 - mean_absolute_error: 0.2792
Epoch 15/1000
22/22 - 5s - lo

In [0]:
model.save('DQ-EP123.h5')

from tensorflow.keras.models import load_model
dqmodel = load_model('DQ-EP123.h5')

In [0]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np 
values = dataset.values
scaler = StandardScaler()
scaled = scaler.fit_transform(values)
X = scaled[:]
y = scaled[:, 0]
timesteps = 864
train_generator = TimeseriesGenerator(X, y, length=timesteps, sampling_rate=1, batch_size=timesteps)
train_X, train_y = train_generator[0]
train_samples = train_X.shape[0]*len(train_generator)



X_test =  your new test data
y_dummy = np.zeros((X_test.shape[0], ))
test_generator = TimeseriesGenerator(X_test, y_dummy, length=timesteps * sampling_rate, sampling_rate=sampling_rate, batch_size=batch_size, shuffle=shuffle_data)
y_test_pred = model.predict_generator(generator=test_generator)